In [128]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pandas as pd

<b> Загружаем данные

In [129]:
data = load_digits()

<b> Разбиваем данные на train и test выборки

In [130]:
images_train, images_val, labels_train, labels_val = train_test_split(data.get('images'), data.get('target'))

<b> Препроцессинг

In [131]:
images = images_train
img_shape = images[0].shape
images = images / tf.reduce_max(images)
images = tf.data.Dataset.from_tensor_slices(images)

labels = labels_train
n_labels = len(set(labels))
labels = tf.keras.utils.to_categorical(labels, num_classes=n_labels)
labels = tf.data.Dataset.from_tensor_slices(labels)

dataset = tf.data.Dataset.zip((images, labels)).batch(4)

In [132]:
images_val = images_val / tf.reduce_max(images_val)
images_val = tf.data.Dataset.from_tensor_slices(images_val)

labels_val = tf.keras.utils.to_categorical(labels_val, num_classes=n_labels)
labels_val = tf.data.Dataset.from_tensor_slices(labels_val)

dataset_val = tf.data.Dataset.zip((images_val, labels_val)).batch(4)

<b> Данные после препроцессинга

In [133]:
it = iter(dataset)
for i in range(3):
    print(next(it))

(<tf.Tensor: shape=(4, 8, 8), dtype=float64, numpy=
array([[[0.    , 0.    , 0.3125, 1.    , 0.875 , 0.125 , 0.    , 0.    ],
        [0.    , 0.0625, 0.8125, 0.875 , 1.    , 0.5   , 0.    , 0.    ],
        [0.    , 0.5625, 0.9375, 0.1875, 1.    , 0.3125, 0.    , 0.    ],
        [0.    , 0.625 , 0.8125, 0.1875, 1.    , 0.1875, 0.    , 0.    ],
        [0.    , 0.1875, 0.1875, 0.6875, 0.8125, 0.    , 0.    , 0.    ],
        [0.    , 0.    , 0.    , 0.8125, 0.625 , 0.    , 0.    , 0.    ],
        [0.    , 0.    , 0.125 , 1.    , 1.    , 1.    , 0.625 , 0.    ],
        [0.    , 0.    , 0.375 , 1.    , 0.875 , 0.75  , 0.5625, 0.    ]],

       [[0.    , 0.    , 0.75  , 1.    , 1.    , 0.9375, 0.1875, 0.    ],
        [0.    , 0.25  , 1.    , 1.    , 0.375 , 0.125 , 0.0625, 0.    ],
        [0.    , 0.875 , 1.    , 0.25  , 0.    , 0.    , 0.    , 0.    ],
        [0.    , 0.5625, 1.    , 1.    , 0.3125, 0.    , 0.    , 0.    ],
        [0.    , 0.    , 0.4375, 0.9375, 0.5   , 0.    , 0

<b> Построим простую однослойную сетку и посмотрим на сколько ее можно обучить

In [141]:
inp = tf.keras.layers.Input(shape = img_shape)
flat = tf.keras.layers.Flatten()(inp)
fc1 = tf.keras.layers.Dense(10, activation='relu')(flat)
out = tf.keras.layers.Softmax()(fc1)
model = tf.keras.models.Model(inputs = inp, outputs = out)
print(model.summary())

Model: "functional_83"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_39 (InputLayer)        [(None, 8, 8)]            0         
_________________________________________________________________
flatten_38 (Flatten)         (None, 64)                0         
_________________________________________________________________
dense_86 (Dense)             (None, 10)                650       
_________________________________________________________________
softmax_41 (Softmax)         (None, 10)                0         
Total params: 650
Trainable params: 650
Non-trainable params: 0
_________________________________________________________________
None


In [142]:
loss = tf.keras.losses.CategoricalCrossentropy()
metrics = ['accuracy']
optimizer = tf.keras.optimizers.Adam(lr = 0.01, clipnorm = 5)
callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                  factor=0.1, 
                                                  patience=5, 
                                                  min_lr=0.0001, 
                                                  min_delta=0.01, 
                                                  cooldown=5)]
model.compile(loss = loss, metrics = metrics)

In [143]:
model.fit(dataset,  epochs = 100, validation_data = dataset_val, callbacks= callbacks)

Epoch 1/100
337/337 [==============================] - 1s 2ms/step - loss: 2.0500 - accuracy: 0.3682 - val_loss: 1.8293 - val_accuracy: 0.5822
Epoch 2/100
337/337 [==============================] - 1s 2ms/step - loss: 1.6694 - accuracy: 0.6125 - val_loss: 1.5328 - val_accuracy: 0.6422
Epoch 3/100
337/337 [==============================] - 1s 2ms/step - loss: 1.4251 - accuracy: 0.6466 - val_loss: 1.3428 - val_accuracy: 0.6533
Epoch 4/100
337/337 [==============================] - 1s 2ms/step - loss: 1.2685 - accuracy: 0.6555 - val_loss: 1.2198 - val_accuracy: 0.6578
Epoch 5/100
337/337 [==============================] - 1s 2ms/step - loss: 1.1651 - accuracy: 0.6630 - val_loss: 1.1368 - val_accuracy: 0.6644
Epoch 6/100
337/337 [==============================] - 1s 2ms/step - loss: 1.0937 - accuracy: 0.6689 - val_loss: 1.0774 - val_accuracy: 0.6689
Epoch 7/100
337/337 [==============================] - 1s 2ms/step - loss: 1.0420 - accuracy: 0.6719 - val_loss: 1.0336 - val_accuracy: 0.6733

Epoch 58/100
337/337 [==============================] - 1s 2ms/step - loss: 0.5526 - accuracy: 0.7862 - val_loss: 0.5843 - val_accuracy: 0.7689
Epoch 59/100
337/337 [==============================] - 1s 2ms/step - loss: 0.5525 - accuracy: 0.7862 - val_loss: 0.5842 - val_accuracy: 0.7689
Epoch 60/100
337/337 [==============================] - 1s 2ms/step - loss: 0.5523 - accuracy: 0.7862 - val_loss: 0.5841 - val_accuracy: 0.7689
Epoch 61/100
337/337 [==============================] - 1s 2ms/step - loss: 0.5522 - accuracy: 0.7862 - val_loss: 0.5840 - val_accuracy: 0.7689
Epoch 62/100
337/337 [==============================] - 1s 2ms/step - loss: 0.5521 - accuracy: 0.7862 - val_loss: 0.5838 - val_accuracy: 0.7689
Epoch 63/100
337/337 [==============================] - 1s 2ms/step - loss: 0.5519 - accuracy: 0.7862 - val_loss: 0.5837 - val_accuracy: 0.7689
Epoch 64/100
337/337 [==============================] - 1s 2ms/step - loss: 0.5518 - accuracy: 0.7862 - val_loss: 0.5836 - val_accuracy:

<b> Смотрим точность на тестовых данных

In [144]:
val_loss, val_acc = model.evaluate(dataset_val)

113/113 [==============================] - 0s 1ms/step - loss: 0.5802 - accuracy: 0.7689


<b> Посмотрим какая точность будет с разными learning rate

In [147]:
alpha = [0.1, 0.01,0.001, 0.0001]
result_acc = []
result_val_acc = []
for i in alpha:
    inp = tf.keras.layers.Input(shape = img_shape)
    flat = tf.keras.layers.Flatten()(inp)
    fc1 = tf.keras.layers.Dense(10, activation='relu')(flat)
    out = tf.keras.layers.Softmax()(fc1)
    model = tf.keras.models.Model(inputs = inp, outputs = out)
    loss = tf.keras.losses.CategoricalCrossentropy()
    metrics = ['accuracy']
    optimizer = tf.keras.optimizers.Adam(lr = 0.01, clipnorm = 5)
    model.compile(loss = loss, metrics = metrics)
    model.fit(dataset,  epochs = 100, validation_data = dataset_val)
    acc = model.evaluate(dataset)[1]
    val_acc = model.evaluate(dataset_val)[1]
    result_acc.append(acc)
    result_val_acc.append(val_acc)

Epoch 1/100
337/337 [==============================] - 1s 2ms/step - loss: 2.2192 - accuracy: 0.1878 - val_loss: 2.1320 - val_accuracy: 0.2489
Epoch 2/100
337/337 [==============================] - 1s 2ms/step - loss: 2.1023 - accuracy: 0.2613 - val_loss: 2.0359 - val_accuracy: 0.3000
Epoch 3/100
337/337 [==============================] - 1s 2ms/step - loss: 2.0227 - accuracy: 0.2903 - val_loss: 1.9724 - val_accuracy: 0.3111
Epoch 4/100
337/337 [==============================] - 1s 2ms/step - loss: 1.9700 - accuracy: 0.3103 - val_loss: 1.9275 - val_accuracy: 0.3244
Epoch 5/100
337/337 [==============================] - 1s 2ms/step - loss: 1.9326 - accuracy: 0.3215 - val_loss: 1.8949 - val_accuracy: 0.3311
Epoch 6/100
337/337 [==============================] - 1s 2ms/step - loss: 1.9052 - accuracy: 0.3318 - val_loss: 1.8705 - val_accuracy: 0.3400
Epoch 7/100
337/337 [==============================] - 1s 2ms/step - loss: 1.8842 - accuracy: 0.3437 - val_loss: 1.8514 - val_accuracy: 0.3467

Epoch 58/100
337/337 [==============================] - 1s 2ms/step - loss: 1.7214 - accuracy: 0.3883 - val_loss: 1.7031 - val_accuracy: 0.3800
Epoch 59/100
337/337 [==============================] - 1s 2ms/step - loss: 1.7207 - accuracy: 0.3883 - val_loss: 1.7026 - val_accuracy: 0.3800
Epoch 60/100
337/337 [==============================] - 1s 2ms/step - loss: 1.7200 - accuracy: 0.3883 - val_loss: 1.7020 - val_accuracy: 0.3800
Epoch 61/100
337/337 [==============================] - 1s 2ms/step - loss: 1.7193 - accuracy: 0.3883 - val_loss: 1.7014 - val_accuracy: 0.3800
Epoch 62/100
337/337 [==============================] - 1s 2ms/step - loss: 1.7187 - accuracy: 0.3883 - val_loss: 1.7009 - val_accuracy: 0.3800
Epoch 63/100
337/337 [==============================] - 1s 2ms/step - loss: 1.7180 - accuracy: 0.3883 - val_loss: 1.7003 - val_accuracy: 0.3800
Epoch 64/100
337/337 [==============================] - 1s 2ms/step - loss: 1.7174 - accuracy: 0.3883 - val_loss: 1.6998 - val_accuracy:

Epoch 14/100
337/337 [==============================] - 1s 2ms/step - loss: 1.3878 - accuracy: 0.5494 - val_loss: 1.3766 - val_accuracy: 0.5667
Epoch 15/100
337/337 [==============================] - 1s 2ms/step - loss: 1.3787 - accuracy: 0.5509 - val_loss: 1.3684 - val_accuracy: 0.5667
Epoch 16/100
337/337 [==============================] - 1s 2ms/step - loss: 1.3707 - accuracy: 0.5546 - val_loss: 1.3611 - val_accuracy: 0.5667
Epoch 17/100
337/337 [==============================] - 1s 2ms/step - loss: 1.3635 - accuracy: 0.5561 - val_loss: 1.3548 - val_accuracy: 0.5689
Epoch 18/100
337/337 [==============================] - 1s 2ms/step - loss: 1.3571 - accuracy: 0.5590 - val_loss: 1.3491 - val_accuracy: 0.5711
Epoch 19/100
337/337 [==============================] - 1s 2ms/step - loss: 1.3512 - accuracy: 0.5605 - val_loss: 1.3443 - val_accuracy: 0.5711
Epoch 20/100
337/337 [==============================] - 1s 2ms/step - loss: 1.3460 - accuracy: 0.5612 - val_loss: 1.3398 - val_accuracy:

Epoch 71/100
337/337 [==============================] - 1s 2ms/step - loss: 1.2683 - accuracy: 0.5754 - val_loss: 1.2785 - val_accuracy: 0.5800
Epoch 72/100
337/337 [==============================] - 1s 2ms/step - loss: 1.2678 - accuracy: 0.5754 - val_loss: 1.2782 - val_accuracy: 0.5800
Epoch 73/100
337/337 [==============================] - 1s 2ms/step - loss: 1.2673 - accuracy: 0.5761 - val_loss: 1.2779 - val_accuracy: 0.5800
Epoch 74/100
337/337 [==============================] - 1s 2ms/step - loss: 1.2668 - accuracy: 0.5768 - val_loss: 1.2775 - val_accuracy: 0.5800
Epoch 75/100
337/337 [==============================] - 1s 2ms/step - loss: 1.2664 - accuracy: 0.5754 - val_loss: 1.2771 - val_accuracy: 0.5800
Epoch 76/100
337/337 [==============================] - 1s 2ms/step - loss: 1.2658 - accuracy: 0.5761 - val_loss: 1.2769 - val_accuracy: 0.5800
Epoch 77/100
337/337 [==============================] - 1s 2ms/step - loss: 1.2654 - accuracy: 0.5761 - val_loss: 1.2767 - val_accuracy:

Epoch 27/100
337/337 [==============================] - 1s 2ms/step - loss: 1.6862 - accuracy: 0.4009 - val_loss: 1.7408 - val_accuracy: 0.3644
Epoch 28/100
337/337 [==============================] - 1s 2ms/step - loss: 1.6847 - accuracy: 0.4009 - val_loss: 1.7397 - val_accuracy: 0.3644
Epoch 29/100
337/337 [==============================] - 1s 2ms/step - loss: 1.6833 - accuracy: 0.4009 - val_loss: 1.7386 - val_accuracy: 0.3644
Epoch 30/100
337/337 [==============================] - 1s 2ms/step - loss: 1.6819 - accuracy: 0.4009 - val_loss: 1.7376 - val_accuracy: 0.3644
Epoch 31/100
337/337 [==============================] - 1s 2ms/step - loss: 1.6806 - accuracy: 0.4009 - val_loss: 1.7366 - val_accuracy: 0.3644
Epoch 32/100
337/337 [==============================] - 1s 2ms/step - loss: 1.6794 - accuracy: 0.4009 - val_loss: 1.7356 - val_accuracy: 0.3644
Epoch 33/100
337/337 [==============================] - 1s 2ms/step - loss: 1.6782 - accuracy: 0.4009 - val_loss: 1.7348 - val_accuracy:

Epoch 84/100
337/337 [==============================] - 1s 2ms/step - loss: 1.6529 - accuracy: 0.4024 - val_loss: 1.7146 - val_accuracy: 0.3600
Epoch 85/100
337/337 [==============================] - 1s 2ms/step - loss: 1.6527 - accuracy: 0.4024 - val_loss: 1.7144 - val_accuracy: 0.3600
Epoch 86/100
337/337 [==============================] - 1s 2ms/step - loss: 1.6525 - accuracy: 0.4024 - val_loss: 1.7142 - val_accuracy: 0.3600
Epoch 87/100
337/337 [==============================] - 1s 2ms/step - loss: 1.6523 - accuracy: 0.4024 - val_loss: 1.7140 - val_accuracy: 0.3600
Epoch 88/100
337/337 [==============================] - 1s 2ms/step - loss: 1.6521 - accuracy: 0.4024 - val_loss: 1.7138 - val_accuracy: 0.3600
Epoch 89/100
337/337 [==============================] - 1s 2ms/step - loss: 1.6518 - accuracy: 0.4031 - val_loss: 1.7136 - val_accuracy: 0.3600
Epoch 90/100
337/337 [==============================] - 1s 2ms/step - loss: 1.6516 - accuracy: 0.4031 - val_loss: 1.7135 - val_accuracy:

Epoch 40/100
337/337 [==============================] - 1s 2ms/step - loss: 1.2968 - accuracy: 0.4863 - val_loss: 1.3280 - val_accuracy: 0.4733
Epoch 41/100
337/337 [==============================] - 1s 2ms/step - loss: 1.2956 - accuracy: 0.4863 - val_loss: 1.3269 - val_accuracy: 0.4733
Epoch 42/100
337/337 [==============================] - 1s 2ms/step - loss: 1.2944 - accuracy: 0.4863 - val_loss: 1.3260 - val_accuracy: 0.4733
Epoch 43/100
337/337 [==============================] - 1s 2ms/step - loss: 1.2934 - accuracy: 0.4863 - val_loss: 1.3251 - val_accuracy: 0.4733
Epoch 44/100
337/337 [==============================] - 1s 2ms/step - loss: 1.2923 - accuracy: 0.4863 - val_loss: 1.3239 - val_accuracy: 0.4733
Epoch 45/100
337/337 [==============================] - 1s 2ms/step - loss: 1.2912 - accuracy: 0.4863 - val_loss: 1.3231 - val_accuracy: 0.4733
Epoch 46/100
337/337 [==============================] - 1s 2ms/step - loss: 1.2903 - accuracy: 0.4863 - val_loss: 1.3222 - val_accuracy:

Epoch 97/100
337/337 [==============================] - 1s 2ms/step - loss: 1.2662 - accuracy: 0.4855 - val_loss: 1.3023 - val_accuracy: 0.4733
Epoch 98/100
337/337 [==============================] - 1s 2ms/step - loss: 1.2659 - accuracy: 0.4855 - val_loss: 1.3021 - val_accuracy: 0.4733
Epoch 99/100
337/337 [==============================] - 1s 2ms/step - loss: 1.2657 - accuracy: 0.4848 - val_loss: 1.3019 - val_accuracy: 0.4733
Epoch 100/100
113/113 [==============================] - 0s 1ms/step - loss: 1.3017 - accuracy: 0.4733


<b> Визуализируем полученные данные

In [148]:
import plotly.graph_objects as go

In [155]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = alpha, y = result_acc,
                    mode='markers',
                    name='Accuracy'))
fig.add_trace(go.Scatter(x = alpha, y = result_val_acc,
                        mode = 'markers',
                        name = 'Val accuracy'))
fig.show()